In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import sunpy.map
from aiapy.calibrate.prep import register as aiaprep  # sunpy.instr.aia.aiaprep was moved to aiapy
from sunpy.net import Fido, attrs as a
from astropy import units as u

import glob
import numpy as np

import warnings
warnings.filterwarnings("ignore")


#https://github.com/tqdm/tqdm/issues/481
#disable the monitor thread
from tqdm import tqdm
tqdm.monitor_interval = 0

In [ ]:
t = a.Time('2016-06-12T07:54:15', '2016-06-12T11:43:38')

folder = '/Users/delbe/OneDrive - FHNW/aia_data/test'
# folder_iris = "../../../../cr_obs3/20140418_123338_3820259153/Si IV 1400"
p1 = [780,2277]  # x,y of p1 (begin of slit in AIA coordinates)
p2 = [781,2180]  # x,y of p2 (end of slit in AIA coordinates)

In [ ]:
# search for aia sources
result = Fido.search(t, a.Instrument("aia"), a.Wavelength(304*u.angstrom), a.Sample(1*u.second))

In [ ]:
# download files, skip if already downloaded
Fido.fetch(result, path=folder)

In [ ]:
# get downloaded files
files = glob.glob(folder + '/*')

In [ ]:
# This should be modified in such a way that it reads the input from the excel sheet and calculates all the images. 

gradient = (p2[1]-p1[1])/(p2[0]-p1[0])
origin = p1[1] - gradient * p1[0]

# x = np.linspace(p1[0], p2[0], 200)  # 200 = slit-size in pixels?
# y = (gradient*x + origin)
# coords = (np.column_stack((x.astype(int),y.astype(int))))

# alternative way to get coordinates - assuming slit is always (almost) vertical
y = np.arange(min(p1[1], p2[1]),max(p1[1], p2[1])+1)
x = (y - origin) / gradient
coords = (np.column_stack((x.round().astype(int),y.astype(int))))

final_image = []

for i in range(0,len(files)-1):
# for i in range(0,100):
    
    if i%10 == 0:
        print(i, end=',')
    
    try:         
        aia1 = sunpy.map.Map(files[i])
    except: 
        continue
        
    aia = aiaprep(aia1)
    
    zi = np.flipud(aia.data.T)[coords[:,1], coords[:,0]]
    
    if len(final_image) == 0: 
        final_image = zi 
    else:
        final_image = np.vstack([final_image, zi])

plt.figure(figsize=[30,10]) # on this image, we should see the downflows.
plt.imshow(final_image.transpose()) # Please adapt the axes for the correct annotation (y-solar coordinates and x-time)

In [ ]:
plt.figure(figsize=(30,10))
plt.imshow(np.log(final_image.transpose()))

In [ ]:
plt.figure(figsize=[45,25]) 
plt.imshow(final_image.transpose()) 
plt.title(aia.date.isot, c = "red")
plt.show()

In [ ]:
# draw AIA context around slit

x, y = (p1[0] + p2[0])//2, (p1[1] + p2[1])//2
dy = abs(p1[1] - p2[1]) * 2

plt.figure(figsize=(12,12))

aia = aiaprep(sunpy.map.Map(files[0]))
plt.subplot(1, 3, 1)
plt.title(aia.date.isot)
im = np.flipud(aia.data.T)[y-dy:y+dy, x-dy:x+dy]
plt.plot([p1[0]-x+dy, p2[0]-x+dy], [p1[1]-y+dy, p2[1]-y+dy], color="red", linewidth=1)
plt.imshow(im)
        
aia = aiaprep(sunpy.map.Map(files[len(files) // 2]))
plt.subplot(1, 3, 2)
plt.title(aia.date.isot)
im = np.flipud(aia.data.T)[y-dy:y+dy, x-dy:x+dy]
plt.plot([p1[0]-x+dy, p2[0]-x+dy], [p1[1]-y+dy, p2[1]-y+dy], color="red", linewidth=1)
plt.imshow(im)

aia = aiaprep(sunpy.map.Map(files[-1]))
plt.subplot(1, 3, 3)
plt.title(aia.date.isot)
im = np.flipud(aia.data.T)[y-dy:y+dy, x-dy:x+dy]
plt.plot([p1[0]-x+dy, p2[0]-x+dy], [p1[1]-y+dy, p2[1]-y+dy], color="red", linewidth=1)
plt.imshow(im)

In [ ]:
plt.figure(figsize=(30,10))
plt.imshow(final_image.transpose())
plt.title(aia.date.isot)
plt.show()
# plt.title(aia1.fits_header['DATE-OBS'])
# plt.title(aia1.date.value)

In [ ]:
aia.date.isot

In [ ]:
print(result[0]["Start Time"])

In [ ]:
print(result[0]["End Time"])

In [ ]:
list_obs = [('2016-06-12T07:54:15', '2016-06-12T11:43:38'),('2014-04-18T12:33:38', '2014-04-18T17:18:01')]

for i in list_obs:
    print(i)